***GENERATED CODE FOR clusteringtest PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


Feature_Transformations_Methods = {

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from datatransformations import VectorAssembler
import json
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.feature import VectorAssembler


def vectorAssemblerTransform(df, param):

    dfReturn = df
    outcol = param + "_vector"
    assembler = VectorAssembler(inputCols=[param], outputCol=outcol)

    dfReturn = assembler.transform(dfReturn)

    return dfReturn


def bkmeans(df, k=3):
    list = df.columns
    new_df = VectorAssembler.vectorAssemblerTransform(df, list)
    bkmeans = BisectingKMeans(k=k)
    model = bkmeans.fit(new_df.select('features'))
    transformed = model.transform(new_df)
    return transformed


class Clustering:

    def run(spark_DF, spark, config):
        stage_attribute = json.loads(config)
        stage_attribute['model']
        bkmeans_obj = bkmeans(df=spark_DF)
        [bkmeans_obj]


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	clusteringtest_DBFS = DBFSConnector.fetch([], {}, "5eb0122b205c0278a02933f4", spark, "{'url': '/Demo/clustering/IrisTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	clusteringtest_AutoFE = TransformationMain.run(clusteringtest_DBFS, spark,json.dumps( {"FE": [{"transformationsData": {}, "feature": "SepalLengthCm", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "98", "mean": "5.78", "stddev": "0.78", "min": "4.4", "max": "7.7", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "SepalWidthCm", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "98", "mean": "3.06", "stddev": "0.47", "min": "2.0", "max": "4.4", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "PetalLengthCm", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "98", "mean": "3.68", "stddev": "1.78", "min": "1.0", "max": "6.9", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "PetalWidthCm", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "98", "mean": "1.18", "stddev": "0.78", "min": "0.1", "max": "2.5", "missing": "0"}, "transformation": ""}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	clusteringtest_Cluster = Clustering.run(clusteringtest_AutoFE,spark,json.dumps( {"autoClustering": 1, "defaultclusters": 1, "orignalfile": "/Demo/clustering/IrisTrain.csv", "distinct_column": "SepalWidthCm", "model": "Bisecting KMeans", "run_id": "a26de165cdde45ae9babc05f47c36e09", "model_id": "5eb0242f1bfdaec91f6d125d"}))

except Exception as ex: 
	logging.error(ex)
